In [2]:
import os
import json
import torch
import numpy as np
from PIL import Image

import math

import numpy as np
import pytest
import torch
import torch.nn.functional as F

device = torch.device("cuda:0")

In [3]:
DATA_PATH = "/home/ubuntu/workdir/gsplat/DATA/"
images_folder = os.path.join(DATA_PATH, "images")
poses_file = os.path.join(DATA_PATH, "transforms.json")

In [4]:
# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = Image.open(img_path).convert('RGB')
        images.append(np.array(img))
    return images

# Function to load poses from a JSON file
def load_poses_from_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    poses = []
    for frame in data['frames']:
        poses.append(np.array(frame['transform_matrix']))
    return poses



In [5]:
# Load images
images = load_images_from_folder(images_folder)
images = np.stack([img[..., :3] for img in images])
images = torch.from_numpy(images).float().to(device) / 255.0

print(f"Loaded {images.shape[0]} images from {images_folder}")

Loaded 929 images from /home/ubuntu/workdir/gsplat/DATA/images


In [6]:

# Load poses
poses = load_poses_from_file(poses_file)
c2ws = torch.tensor(poses).float().to(device)


/tmp/ipykernel_40592/29799323.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  c2ws = torch.tensor(poses).float().to(device)


In [7]:

# Example intrinsic matrix (K)
K = torch.tensor([[240.0, 0.0, 240.0], [0.0, 180.0, 180.0], [0.0, 0.0, 1.0]]).to(device)  # Adjust this based on your actual camera intrinsics

# Dummy data for points and colors (you should replace these with your actual data)
points = torch.rand(5000, 3).float().to(device)
points_rgb = torch.randint(0, 255, (5000, 3)).float().to(device)

# Number of images and their dimensions
n_images = len(images)
height, width = images.shape[1:3]
